In [1]:
import findspark
findspark.init()
import pandas as pd
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
spark = SparkSession.builder \
    .master('spark://172.12.0.2:7077') \
    .config("spark.driver.memory", "6g")\
    .config('spark.cores.max', '4')\
    .appName('myapp1') \
    .enableHiveSupport() \
    .getOrCreate()
# code run spark job to load parquetfile in HDFS

21/12/16 15:48:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/16 15:48:37 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [2]:
#read parquet file in HDFS. 
parquetFileDF = spark.read.parquet("/user/input/dataset_parquet/part.0.parquet")

In [3]:
# show data from parquet file
parquetFileDF.show()

+----+--------+--------------------+--------------------+-------------+----------------+--------------------+------------+-------+
|  id|composer|         composition|            movement|     ensemble|          source|         transcriber|catalog_name|seconds|
+----+--------+--------------------+--------------------+-------------+----------------+--------------------+------------+-------+
|1727|Schubert|Piano Quintet in ...|          2. Andante|Piano Quintet|European Archive|http://tirolmusic...|       OP114|    447|
|1728|Schubert|Piano Quintet in ...|  3. Scherzo: Presto|Piano Quintet|European Archive|http://tirolmusic...|       OP114|    251|
|1729|Schubert|Piano Quintet in ...|4. Andantino - Al...|Piano Quintet|European Archive|http://tirolmusic...|       OP114|    444|
|1730|Schubert|Piano Quintet in ...|   5. Allegro giusto|Piano Quintet|European Archive|http://tirolmusic...|       OP114|    368|
|1733|Schubert|Piano Sonata in A...|        2. Andantino|   Solo Piano|        Muse

In [11]:

# function map : tạo lên 1 cặp key - value: key = 1, value = composer
def map(parquetFileDF):
    df=parquetFileDF.toPandas()
    map_data = pd.DataFrame()
    for line in range(df.count()[0]):
        map_data.at[line,'composer'] = df.at[line,'composer']
        map_data.at[line,'value'] = 1
    return map_data
    
def reducer(mapdata):
    current_composer = None
    current_count = 0
    composer = None
    df_output = pd.DataFrame()
    line=None
    for line in range(mapdata.count()[0]):
        composer = mapdata.at[line,'composer']
        count = mapdata.at[line,'value']
        try:
            count = int(count)
        except ValueError:
            # count was not a number, so silently
            # ignore/discard this line
            continue
        if current_composer == composer:
            current_count += count
        else:
            if current_composer:
                df_output.at[line,'composer'] = current_composer
                df_output.at[line,'count'] = current_count
            current_count = count
            current_composer = composer

    if current_composer == composer:
        df_output.at[line+1,'composer'] = current_composer
        df_output.at[line+1,'count'] = current_count
    return df_output

mapdata_ = map(parquetFileDF)
mapdata_ = mapdata_.sort_values(by='composer', ascending=True).reset_index()
print(mapdata_)
output = reducer(mapdata_)
print(output.values)

     index  composer  value
0      164      Bach    1.0
1      129      Bach    1.0
2      130      Bach    1.0
3      131      Bach    1.0
4      132      Bach    1.0
..     ...       ...    ...
325     27  Schubert    1.0
326     28  Schubert    1.0
327     29  Schubert    1.0
328     15  Schubert    1.0
329      0  Schubert    1.0

[330 rows x 3 columns]
[['Bach' 67.0]
 ['Beethoven' 157.0]
 ['Brahms' 24.0]
 ['Cambini' 9.0]
 ['Dvorak' 8.0]
 ['Faure' 4.0]
 ['Haydn' 3.0]
 ['Mozart' 24.0]
 ['Ravel' 4.0]
 ['Schubert' 30.0]]


In [12]:
df1 = spark.createDataFrame(output.astype(str))
df1.write.parquet("/user/output/output.parquet")
# read output
output = spark.read.parquet("/user/output/output.parquet")
output.show()

+---------+-----+
| composer|count|
+---------+-----+
|    Faure|  4.0|
|    Haydn|  3.0|
|   Mozart| 24.0|
|    Ravel|  4.0|
| Schubert| 30.0|
|     Bach| 67.0|
|Beethoven|157.0|
|   Brahms| 24.0|
|  Cambini|  9.0|
|   Dvorak|  8.0|
+---------+-----+

